# Loan Default Prediction

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

from sklearn.feature_selection import RFE
from statsmodels import api as sm
from dateutil.parser import parse

In [2]:
trainperf = pd.read_csv('trainperf.csv', sep=',')
traindemo = pd.read_csv('traindemographics.csv', sep=',')
testperf = pd.read_csv('testperf.csv', sep=',')
testdemo = pd.read_csv('testdemographics.csv', sep=',')
trainprev = pd.read_csv('trainprevloans/trainprevloans.csv', sep=',')
testprev = pd.read_csv('testprevloans/testprevloans.csv', sep=',')

In [3]:
trainperf.sample(5)

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,referredby,good_bad_flag
584,8a858ef7599330e0015993c3e1af2ecc,301999779,7,2017-07-28 03:01:12.000000,2017-07-28 02:00:07.000000,20000.0,24500.0,30,NaN,Good
2086,8a85886e5410189001541480d8f83683,301990332,12,2017-07-22 07:30:00.000000,2017-07-22 06:29:44.000000,40000.0,44000.0,30,NaN,Good
3680,8a858fe758dc437a0158eb12515362d4,301988346,5,2017-07-20 21:17:57.000000,2017-07-20 20:17:52.000000,20000.0,24500.0,30,NaN,Good
2782,8a858e725c3ae262015c3b08d771135e,301988873,3,2017-07-21 09:05:17.000000,2017-07-21 08:05:11.000000,10000.0,11500.0,15,8a858e6d57903422015798efba102d26,Good
1219,8a8589b953d0be470153d166c59f388e,301958567,8,2017-07-01 06:33:45.000000,2017-07-01 05:33:37.000000,20000.0,23000.0,30,NaN,Good


In [4]:
traindemo.sample(5)

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,NaN,Permanent,NaN
3670,8a858eb75c21a2b9015c2aff4a5f448a,1978-04-14 00:00:00.000000,Savings,7.390357,9.166987,GT Bank,NaN,Permanent,NaN
259,8a858f0b5b439ebb015b43cf463d3e39,1988-07-21 00:00:00.000000,Savings,8.897512,9.839003,Stanbic IBTC,NaN,NaN,NaN
975,8a858e3d570f30bb0157146c32c418b7,1994-04-26 00:00:00.000000,Savings,5.628345,6.407138,GT Bank,NaN,Student,NaN
2411,8a858fa154e2912f0155112438744810,1983-02-10 00:00:00.000000,Savings,3.076108,6.520516,Stanbic IBTC,NaN,Permanent,NaN


In [5]:
testprev.sample(3)

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,closeddate,referredby,firstduedate,firstrepaiddate
4750,8a858faf55d95d930155f02fd7f92a5a,301879972,11,2017-04-25 13:38:30.000000,2017-04-25 12:38:23.000000,40000.0,44800.0,30,2017-05-24 09:03:06.000000,NaN,2017-05-25 00:00:00.000000,2017-05-24 08:47:57.000000
5753,8a858f1554e29449015505ceef786df0,301942394,17,2017-06-17 23:01:20.000000,2017-06-17 22:01:13.000000,40000.0,44000.0,30,2017-06-21 16:31:14.000000,NaN,2017-07-17 00:00:00.000000,2017-06-21 16:21:05.000000
589,8a858f9d5afc0a16015afc552c271b88,301845225,1,2017-03-23 20:02:56.000000,2017-03-23 19:02:40.000000,10000.0,11500.0,15,2017-03-31 14:54:15.000000,NaN,2017-04-07 00:00:00.000000,2017-03-31 14:39:06.000000


In [6]:
trainprev.isnull().sum()

customerid             0
systemloanid           0
loannumber             0
approveddate           0
creationdate           0
loanamount             0
totaldue               0
termdays               0
closeddate             0
referredby         17157
firstduedate           0
firstrepaiddate        0
dtype: int64

In [7]:
trainprev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18183 entries, 0 to 18182
Data columns (total 12 columns):
customerid         18183 non-null object
systemloanid       18183 non-null int64
loannumber         18183 non-null int64
approveddate       18183 non-null object
creationdate       18183 non-null object
loanamount         18183 non-null float64
totaldue           18183 non-null float64
termdays           18183 non-null int64
closeddate         18183 non-null object
referredby         1026 non-null object
firstduedate       18183 non-null object
firstrepaiddate    18183 non-null object
dtypes: float64(2), int64(3), object(7)
memory usage: 1.7+ MB


In [8]:
trainperf.isnull().sum()

customerid          0
systemloanid        0
loannumber          0
approveddate        0
creationdate        0
loanamount          0
totaldue            0
termdays            0
referredby       3781
good_bad_flag       0
dtype: int64

In [9]:
testperf.isnull().sum()

customerid         0
systemloanid       0
loannumber         0
approveddate       0
creationdate       0
loanamount         0
totaldue           0
termdays           0
referredby      1266
dtype: int64

In [10]:
traindemo.isnull().sum()

customerid                       0
birthdate                        0
bank_account_type                0
longitude_gps                    0
latitude_gps                     0
bank_name_clients                0
bank_branch_clients           4295
employment_status_clients      648
level_of_education_clients    3759
dtype: int64

In [12]:
testdemo.isnull().sum()

customerid                       0
birthdate                        0
bank_account_type                0
longitude_gps                    0
latitude_gps                     0
bank_name_clients                0
bank_branch_clients           1473
employment_status_clients      217
level_of_education_clients    1277
dtype: int64

In [11]:
trainperf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4368 entries, 0 to 4367
Data columns (total 10 columns):
customerid       4368 non-null object
systemloanid     4368 non-null int64
loannumber       4368 non-null int64
approveddate     4368 non-null object
creationdate     4368 non-null object
loanamount       4368 non-null float64
totaldue         4368 non-null float64
termdays         4368 non-null int64
referredby       587 non-null object
good_bad_flag    4368 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 341.3+ KB


In [14]:
traindemo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4346 entries, 0 to 4345
Data columns (total 9 columns):
customerid                    4346 non-null object
birthdate                     4346 non-null object
bank_account_type             4346 non-null object
longitude_gps                 4346 non-null float64
latitude_gps                  4346 non-null float64
bank_name_clients             4346 non-null object
bank_branch_clients           51 non-null object
employment_status_clients     3698 non-null object
level_of_education_clients    587 non-null object
dtypes: float64(2), object(7)
memory usage: 305.7+ KB


In [13]:
testdemo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1487 entries, 0 to 1486
Data columns (total 9 columns):
customerid                    1487 non-null object
birthdate                     1487 non-null object
bank_account_type             1487 non-null object
longitude_gps                 1487 non-null float64
latitude_gps                  1487 non-null float64
bank_name_clients             1487 non-null object
bank_branch_clients           14 non-null object
employment_status_clients     1270 non-null object
level_of_education_clients    210 non-null object
dtypes: float64(2), object(7)
memory usage: 104.6+ KB


In [17]:
traindemo['bank_account_type'].unique()

array(['Savings', 'Other', 'Current'], dtype=object)

In [16]:
traindemo['employment_status_clients'].unique()

array([nan, 'Permanent', 'Student', 'Self-Employed', 'Unemployed',
       'Retired', 'Contract'], dtype=object)

In [15]:
px.histogram(traindemo, x='employment_status_clients')

In [18]:
px.histogram(traindemo, x='level_of_education_clients')

In [19]:
#Drop the columns with missing values
trainperf.drop(labels='referredby', axis=1, inplace=True)
testperf.drop(labels='referredby', axis=1, inplace=True)

trainprev.drop(labels='referredby', axis=1, inplace=True)
testprev.drop(labels='referredby', axis=1, inplace=True)

testdemo.drop(columns=['level_of_education_clients','employment_status_clients','bank_branch_clients'],inplace=True)
traindemo.drop(columns=['level_of_education_clients','employment_status_clients','bank_branch_clients'],inplace=True)

In [20]:
# Convert all date objects into the datetime type
traindemo['birthdate'] = pd.to_datetime(traindemo['birthdate'], infer_datetime_format=True)
testdemo['birthdate'] = pd.to_datetime(testdemo['birthdate'], infer_datetime_format=True)

trainperf['approveddate'] = pd.to_datetime(trainperf['approveddate'], infer_datetime_format=True)
trainperf['creationdate'] = pd.to_datetime(trainperf['creationdate'], infer_datetime_format=True)

In [21]:
trainprev['approveddate'] = trainprev['approveddate'].apply(lambda d: parse(d))
trainprev['creationdate'] = trainprev['creationdate'].apply(lambda d: parse(d))
trainprev['closeddate'] = trainprev['closeddate'].apply(lambda d: parse(d))
trainprev['firstduedate'] = trainprev['firstduedate'].apply(lambda d: parse(d))
trainprev['firstrepaiddate'] = trainprev['firstrepaiddate'].apply(lambda d: parse(d))

testprev['approveddate'] = testprev['approveddate'].apply(lambda d: parse(d))
testprev['creationdate'] = testprev['creationdate'].apply(lambda d: parse(d))
testprev['closeddate'] = testprev['closeddate'].apply(lambda d: parse(d))
testprev['firstduedate'] = testprev['firstduedate'].apply(lambda d: parse(d))
testprev['firstrepaiddate'] = testprev['firstrepaiddate'].apply(lambda d: parse(d))

In [22]:
# Convert object variables to categorical variables
traindemo['bank_account_type'] = traindemo['bank_account_type'].astype('category')
testdemo['bank_account_type'] = testdemo['bank_account_type'].astype('category')

trainperf['good_bad_flag'] = trainperf['good_bad_flag'].astype('category')

In [23]:
#Drop the approveddate and creation date columns for their inconsistent time values
testperf.drop(columns=['approveddate','creationdate'], inplace=True)

In [24]:
testperf.sample(3)

,customerid,systemloanid,loannumber,loanamount,totaldue,termdays
631,8a858ff15b5d3f84015b5d6327223863,301995072,4,20000,24500.0,30
966,8a858f2c59bc3fdd0159c69c632b7d9c,301982229,5,10000,11500.0,15
1424,8a858f305c8dd672015c9b6de9735c21,302001011,2,10000,11500.0,15


In [36]:
traindemo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4346 entries, 0 to 4345
Data columns (total 6 columns):
customerid           4346 non-null object
birthdate            4346 non-null datetime64[ns]
bank_account_type    4346 non-null category
longitude_gps        4346 non-null float64
latitude_gps         4346 non-null float64
bank_name_clients    4346 non-null object
dtypes: category(1), datetime64[ns](1), float64(2), object(2)
memory usage: 174.2+ KB


In [26]:
trainprev.sample()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,closeddate,firstduedate,firstrepaiddate
9171,8a858fa55acd8061015ad15eb8b47940,301866435,2,2017-04-12 22:00:40,2017-04-12 21:00:33,10000.0,13000.0,30,2017-05-11 13:55:05,2017-05-12,2017-05-10


After going through the data cleaning, merge the data into 2 distinct; train and test files. 

In [38]:
px.scatter(traindemo, x='longitude_gps', y='latitude_gps')

In [40]:
BBox = (traindemo.longitude_gps.min(), traindemo.longitude_gps.max(), traindemo.latitude_gps.min(), traindemo.latitude_gps.max())
BBox

(-118.24700929999999,
 151.20928999999998,
 -33.868818299999994,
 71.22806939368171)

In [44]:
nigeria_map = plt.imread('')
fig, ax = plt.subplots(figsize=(10,8))
plt.scatter(traindemo.longitude_gps, traindemo.latitude_gps, zorder=1, alpha=0.2, c='b', s=10)
plt.xlim(BBox[0], BBox[1])
plt.ylim(BBox[2], BBox[3])
plt.imshow(nigeria_map, zorder=0,extent=BBox, aspect='equal')

TypeError: imshow() missing 1 required positional argument: 'X'